In [1]:
import os
from collections import defaultdict
from pprint import pprint

import numpy as np

from pyammanalysis.amm_arb import graph_arbitrage, UniV3Scraper

# Vertex Degrees
In graph theory, the degree of a vertex of a graph is the number of edges that are incident to the vertex. In the Uniswap V3 context, the "degree" of a token is the number of other tokens it has trading pools with.

In [2]:
# get top 1000 pairs
scraper = UniV3Scraper(0)  # TODO: replace dummy block number
top_uniswapv3_pairs = scraper.top_uniswapv3_pairs()["pools"]

In [3]:
flattened_tokens_in_pools = np.concatenate(
    [[x["token0"], x["token1"]] for x in top_uniswapv3_pairs]
)
unique, counts = np.unique(flattened_tokens_in_pools, return_counts=True)
token_count = dict(zip(unique, counts))

In [4]:
def token_count_summary(min_degree: int):
    filtered_dict = {k: v for k, v in token_count.items() if v > min_degree}
    print(f"Number of tokens with at least {min_degree} pools: {len(filtered_dict)}")

In [5]:
for i in range(2, 6):
    token_count_summary(i)

Number of tokens with at least 2 pools: 98
Number of tokens with at least 3 pools: 56
Number of tokens with at least 4 pools: 34
Number of tokens with at least 5 pools: 18


In [6]:
top_uniswapv3_tokens = scraper.top_uniswapv3_tokens()["tokens"]
token_symbols_dict = defaultdict(
    lambda: "oof", {x["id"]: x["symbol"] for x in top_uniswapv3_tokens}
)

In [7]:
pprint({token_symbols_dict[k]: v for k, v in token_count.items() if v >= 3})

{'1INCH': 5,
 'AAVE': 4,
 'ACH': 3,
 'ADS': 3,
 'APE': 5,
 'AVINOC': 5,
 'BADGER': 3,
 'BAT': 3,
 'BUSD': 4,
 'CEL': 5,
 'CGG': 3,
 'CHZ': 4,
 'COMP': 4,
 'CRV': 4,
 'CVX': 4,
 'DAI': 43,
 'DYDX': 4,
 'ELON': 3,
 'ENS': 5,
 'EURT': 5,
 'FEI': 10,
 'FPIS': 5,
 'FRAX': 15,
 'FTX Token': 4,
 'FUN': 9,
 'FXS': 4,
 'GALA': 5,
 'GF': 3,
 'GNO': 3,
 'GUSD': 3,
 'HDRN': 3,
 'HEX': 10,
 'HOP': 3,
 'HOT': 3,
 'ICHI': 8,
 'INST': 3,
 'LEASH': 3,
 'LINK': 6,
 'LOOKS': 5,
 'LQTY': 3,
 'LRC': 5,
 'LUSD': 5,
 'LYXe': 3,
 'MATIC': 3,
 'MIM': 5,
 'MKR': 4,
 'MM': 4,
 'MTA': 3,
 'NCR': 3,
 'NEXO': 3,
 'NII': 4,
 'NU': 4,
 'OCEAN': 3,
 'PAX': 5,
 'PDT': 3,
 'QNT': 3,
 'RAI': 6,
 'RAIL': 5,
 'RARE': 3,
 'RBN': 3,
 'RNG': 9,
 'RPL': 3,
 'SAND': 4,
 'SHIB': 3,
 'SLP': 3,
 'SNX': 6,
 'SOS': 3,
 'SPELL': 3,
 'STG': 4,
 'Silo': 3,
 'TONCOIN': 4,
 'TRIBE': 4,
 'TRU': 3,
 'TUSD': 3,
 'UNI': 7,
 'UOS': 3,
 'USDC': 198,
 'USDT': 96,
 'UST': 4,
 'WBTC': 32,
 'WETH': 590,
 'WRLD': 3,
 'XMT': 3,
 'XSGD': 4,
 'XYO': 6

In [8]:
if not os.path.exists("data"):
    os.makedirs("data")

adjacency_matrix = scraper.create_adj_matrix("data/adjacency_matrix.csv")

C:\Users\user\Documents\GitHub\eth-uniswap-prelim-analysis\pyammanalysis\amm_arb\uniswapv3_scraper.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[token1][token0] = pool["token0Price"]
C:\Users\user\Documents\GitHub\eth-uniswap-prelim-analysis\pyammanalysis\amm_arb\uniswapv3_scraper.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[token0][token1] = pool["token1Price"]


In [9]:
adjacency_matrix.iloc[:5, :5]

,UMIIE COIN,USD Coin,Wrapped Ether,Dai Stablecoin,Tether USD
UMIIE COIN,NaN,NaN,NaN,NaN,NaN
USD Coin,NaN,NaN,1038.005581818182189039817942138576,1.000058140097605097679556042004612,0.9900997650809679204247448292982965
Wrapped Ether,NaN,0.0009633859562184519405305285953405791,NaN,0.0009603296831547446887394695983661049,0.0009796750078125919103282044561790814
Dai Stablecoin,NaN,0.9999418632824693339933589335207062,1041.309060358246735119686546342842,NaN,0.9985005452922931299262415203730471
Tether USD,NaN,1.009999229641492201288760108310844,1020.74666805351043832972699856535,1.001501706448510683182143753147069,NaN


In [10]:
graph_arbitrage.find_arbitrage("data/adjacency_matrix.csv", find_all=True)

c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\eth-uniswap-prelim-analysis-3WCyaTCY-py3.8\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


ARBITRAGE FOUND

Path: ['USD Coin', 'Nahmii', 'Wrapped BTC', 'renBTC', 'Rail', 'Wrapped Ether', 'Celsius', 'USD Coin']
2.7e+119%

Path: ['Wrapped Ether', 'Celsius', 'USD Coin', 'Nahmii', 'Wrapped BTC', 'renBTC', 'Rail', 'Wrapped Ether']
2.7e+119%

Path: ['Wrapped BTC', 'renBTC', 'Rail', 'Wrapped Ether', 'Celsius', 'USD Coin', 'Nahmii', 'Wrapped BTC']
2.7e+119%

Path: ['Celsius', 'USD Coin', 'Nahmii', 'Wrapped BTC', 'renBTC', 'Rail', 'Wrapped Ether', 'Celsius']
2.7e+119%



[['USD Coin',
  'Nahmii',
  'Wrapped BTC',
  'renBTC',
  'Rail',
  'Wrapped Ether',
  'Celsius',
  'USD Coin'],
 ['Wrapped Ether',
  'Celsius',
  'USD Coin',
  'Nahmii',
  'Wrapped BTC',
  'renBTC',
  'Rail',
  'Wrapped Ether'],
 ['Wrapped BTC',
  'renBTC',
  'Rail',
  'Wrapped Ether',
  'Celsius',
  'USD Coin',
  'Nahmii',
  'Wrapped BTC'],
 ['Celsius',
  'USD Coin',
  'Nahmii',
  'Wrapped BTC',
  'renBTC',
  'Rail',
  'Wrapped Ether',
  'Celsius']]